In [ ]:
import numpy as np
import pandas as pd
from pandas import dataframe as df
import geopandas as gp
from gp import Geoseries as gs
from geopy import great_circle
import folium
import geodatasets
import geoplot as gplt
import matplotlib as plt
from shapely.geometry import Point, Polygon

In [ ]:
cleaned_data = "/Users/samanthabastien/Desktop/Desktop - Samantha’s MacBook Pro/fall 2023/tastebudz/repository/tastebudz/dataset/archive/cleaned_data.csv"
test_data = "/Users/samanthabastien/Desktop/Desktop - Samantha’s MacBook Pro/fall 2023/tastebudz/repository/tastebudz/dataset/archive/test_data.csv"


In [ ]:
#only going focus on location and cuisine type for bag of words matrix and tf-idf
df1 = pd.read_csv(cleaned_data)
geo_df = df1.loc("id","name", "category", "lat", "lng")
bow_df = df1.loc("category")

In [ ]:
geometry = gp.points_from_xy(df.lat, df.lng)
geo_df = gp.GeoDataFrame(geo_df[["id","name", "category", "lat", "lng"]], geometry=geometry)

In [ ]:
#map for projection
world = geopandas.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip")

In [ ]:
def create_geofence(lat, long, distance):
    # create boundary
    center_point = Point(long, lat)
    north_point = great_circle(miles=distance).destination(center_point, 0)
    south_point = great_circle(miles=distance).destination(center_point, 180)
    east_point = great_circle(miles=distance).destination(center_point, 90)
    west_point = great_circle(miles=distance).destination(center_point, 270)

    # Create a polygon from the bounding box coordinates
    geofence_polygon = Polygon([north_point, east_point, south_point, west_point, north_point])

    # Create a GeoDataFrame from the polygon
    geofence_gdf = gp.GeoDataFrame(geometry=[geofence_polygon])

    return geofence_gdf

In [ ]:
def plot_geofence(geofence_gdf):
     # Create a world map
    world = gp.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Plot the world map
    ax = gplt.polyplot(world, projection=gcrs.PlateCarree(), figsize=(12, 8))

    # Overlay the geofence polygon
    gplt.polyplot(geofence_gdf, ax=ax, facecolor="none", edgecolor="red")

    # Set plot title
    plt.title("Geofence on World Map")

    # Show the plot
    plt.show()
    return